# EconMLを使って効果を測定する

In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from IPython.display import display, display_markdown, Markdown
from textwrap import dedent

from econml.metalearners import TLearner, XLearner, SLearner
from econml.dr import DRLearner
from econml.dml import CausalForestDML

from econml.dr import ForestDRLearner, LinearDRLearner

from lightgbm import LGBMClassifier

from sklearn.linear_model import LassoCV, HuberRegressor, \
    LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, \
    GradientBoostingRegressor, GradientBoostingClassifier

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Lalando-LoadData-RLang.ipynbで作成したCSVファイルを読み込む

In [2]:
data1 = pd.read_csv('cps1_nsw_data_R.csv')
data3 = pd.read_csv('cps3_nsw_data_R.csv')
data0 = pd.read_csv('nsw_dw_R.csv')

data0['treat'].value_counts()

0    260
1    185
Name: treat, dtype: int64

## ATE/ATT/ATUを計算する関数を定義

In [3]:


def calc_effects(learner, X, z, y, test_size=.25, ite_percentile=[10, 25, 50, 75, 90]):
    X_train, X_test, z_train, z_test, y_train, y_test = \
        train_test_split(X, z, y, test_size=test_size, random_state=0)
    if isinstance(learner, TLearner) or \
        isinstance(learner, XLearner) or \
        isinstance(learner, SLearner):
        learner.fit(Y=y_train, T=z_train, X=X_train)
        ite_test = learner.effect(X_test)
        ite_train = learner.effect(X_train)
    else:
        learner.fit(Y=y_train, T=z_train, X=X_train, W=X_train)
        ite_test = learner.effect(X_test)
        ite_train = learner.effect(X_train)    
    
    index_re75_0_train = np.where(X_train['re75'] == 0)[0]
    index_re75_0_test = np.where(X_test['re75'] == 0)[0]
    
    index_re75_ne0_train = np.where(X_train['re75'] != 0)[0]
    index_re75_ne0_test = np.where(X_test['re75'] != 0)[0]
    
    index_treat = np.where(z_test == 1)[0]
    index_control = np.where(z_test == 0)[0]

    index_treat_train = np.where(z_train == 1)[0]
    index_control_train = np.where(z_train == 0)[0]
    
    ate_test_calc = np.mean(ite_test)
    ate_train_calc = np.mean(ite_train)
    att_test_calc = np.mean(ite_test[index_treat])
    atu_test_calc = np.mean(ite_test[index_control])
    att_train_calc = np.mean(ite_train[index_treat_train])
    atu_train_calc = np.mean(ite_train[index_control_train])
    
    cate_re75_0_train   = np.mean(ite_train[index_re75_0_train])
    cate_re75_0_test    = np.mean(ite_test[index_re75_0_test])
    cate_re75_ne0_train = np.mean(ite_train[index_re75_ne0_train])
    cate_re75_ne0_test  = np.mean(ite_test[index_re75_ne0_test])
    
    
    train_vals = [ate_train_calc, att_train_calc, atu_train_calc, cate_re75_0_train, cate_re75_ne0_train] 
    test_vals  = [ate_test_calc,  att_test_calc,  atu_test_calc,  cate_re75_0_test,  cate_re75_ne0_test]
    
    index      = ['ATE', 'ATT', 'ATU', 'CATE(re75=0)', 'CATE(re75!=0)']
    
    ite_q_train = np.percentile(ite_train, q=ite_percentile)
    ite_q_test  = np.percentile(ite_test, q=ite_percentile)
    
    for i, q in enumerate(ite_percentile):
        train_vals.append(ite_q_train[i])
        test_vals.append(ite_q_test[i])
        index.append(f'ITE({q}%)')
        
    data = pd.DataFrame(dict(train=train_vals, test=test_vals), index=index)
    return data
    
    

## DR(Double Robust)Learner

In [4]:


def run_dr(X, z, y, outcome_model, ps_model):
    # cvというパラメータがあるので、交差検証が使われてるよう
    #dr_learner = DRLearner()
    #learner = DRLearner(model_regression=outcome_model, model_propensity=ps_model)
    learner = ForestDRLearner(model_regression=outcome_model, model_propensity=ps_model)
    
    #learner = LinearDRLearner(random_state=0)
    
    return calc_effects(learner=learner, X=X, y=y, z=z)


### 実行

In [5]:
data = data1.copy()

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

ps_model = LinearDiscriminantAnalysis()

outcome_model = GradientBoostingRegressor(
    alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


#outcome_model = LinearRegression()
#ps_model = LogisticRegression()

run_dr(X=X, y=y, z=z, outcome_model=outcome_model, ps_model=ps_model)


,train,test
ATE,-19211.711479,-19342.097234
ATT,-15678.087537,1927.294085
ATU,-19252.368483,-19592.137603
CATE(re75=0),23654.001774,6166.259101
CATE(re75!=0),-24830.692724,-22559.047518
ITE(10%),-34293.111006,-33447.288953
ITE(25%),-2486.172634,-2459.552044
ITE(50%),-1678.896240,-1673.334564
ITE(75%),-521.559372,-558.677879
ITE(90%),10840.574336,11067.527546


## T-Leaner

In [6]:
def run_t(X, z, y, outcome_model):
    learner = TLearner(models=outcome_model)
    return calc_effects(learner=learner, X=X, y=y, z=z)


### 実行

In [7]:
data = data1.copy()

outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_t(X=X, y=y, z=z, outcome_model=outcome_model)


,train,test
ATE,-6819.458268,-6839.214444
ATT,876.294279,1275.445679
ATU,-6908.003695,-6934.609398
CATE(re75=0),2074.837231,2227.934496
CATE(re75!=0),-7985.352308,-7982.705110
ITE(10%),-14916.139734,-14860.979969
ITE(25%),-12309.331487,-12209.913236
ITE(50%),-7552.038874,-7674.670208
ITE(75%),-1664.584714,-1898.758703
ITE(90%),2095.262639,2051.705751


## X-Learner

In [8]:

def run_x(X, z, y, outcome_model, ps_model):
    #learner = XLearner(models=outcome_model)
    #propensity_model=ps_model,
    #cate_models=GradientBoostingRegressor())

    learner = XLearner(models=outcome_model, propensity_model=ps_model)
            
    return calc_effects(learner=learner, X=X, y=y, z=z)
    
    

### 実行

In [9]:
data = data1.copy()

outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

ps_model = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=4,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.05, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=4,
                           min_weight_fraction_leaf=0.0, n_estimators=180,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=6546, subsample=0.55, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_x(X=X, y=y, z=z, outcome_model=outcome_model, ps_model=ps_model)


,train,test
ATE,-3004.362179,-3046.076429
ATT,1684.940606,1434.349266
ATU,-3058.316138,-3098.747767
CATE(re75=0),3065.853391,3066.841497
CATE(re75!=0),-3800.066364,-3816.998428
ITE(10%),-7272.660600,-7384.230730
ITE(25%),-5466.572708,-5466.123998
ITE(50%),-4118.318473,-4065.047374
ITE(75%),-1194.113678,-1301.318614
ITE(90%),2978.931976,2604.902703


### 実行（線形回帰）
- EconMLとCausalMLの結果が一致することを確認するために用意

In [10]:
data = data1.copy()

outcome_model = LinearRegression()
ps_model = LogisticRegression()

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_x(X=X, y=y, z=z, outcome_model=outcome_model, ps_model=ps_model)


,train,test
ATE,-5542.932862,-5648.363535
ATT,715.186453,-239.834748
ATU,-5614.937237,-5711.945539
CATE(re75=0),4649.428497,4458.659091
CATE(re75!=0),-6878.981722,-6922.996399
ITE(10%),-14122.603764,-14225.199348
ITE(25%),-11486.174951,-11385.929459
ITE(50%),-6383.612245,-6509.551113
ITE(75%),-159.180128,-392.271932
ITE(90%),4320.178439,3997.668819


## S-Learner

In [11]:
def run_s(X, z, y, outcome_model):
    learner = SLearner(overall_model=outcome_model)
    return calc_effects(learner=learner, X=X, y=y, z=z)


### 実行

In [12]:
data = data1.copy()

outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_s(X=X, y=y, z=z, outcome_model=outcome_model)


,train,test
ATE,-1800.104445,-1829.726639
ATT,217.893059,225.825749
ATU,-1823.323025,-1853.891462
CATE(re75=0),1095.627456,1110.322585
CATE(re75!=0),-2179.686680,-2200.506789
ITE(10%),-3596.000761,-3705.252527
ITE(25%),-3159.357001,-3162.993347
ITE(50%),-2087.244132,-2084.859178
ITE(75%),-770.906864,-827.118852
ITE(90%),523.415062,444.815852


### 実行（線形回帰）
- EconMLとCausalMLの結果が一致することを確認するために用意

In [13]:
data = data1.copy()

outcome_model = LinearRegression()

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_s(X=X, y=y, z=z, outcome_model=outcome_model)


,train,test
ATE,720.289796,720.289796
ATT,720.289796,720.289796
ATU,720.289796,720.289796
CATE(re75=0),720.289796,720.289796
CATE(re75!=0),720.289796,720.289796
ITE(10%),720.289796,720.289796
ITE(25%),720.289796,720.289796
ITE(50%),720.289796,720.289796
ITE(75%),720.289796,720.289796
ITE(90%),720.289796,720.289796


## DoubleMachineLearning

In [14]:
def run_dml(X, z, y, outcome_model, ps_model):
    learner = CausalForestDML(model_y=outcome_model,
              model_t=ps_model)
    return calc_effects(learner=learner, X=X, y=y, z=z)

### 実行

In [15]:
data = data1.copy()

outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

ps_model = LinearDiscriminantAnalysis()

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_dml(X=X, y=y, z=z, outcome_model=outcome_model, ps_model=ps_model)



,train,test
ATE,77.114765,36.351293
ATT,1418.333449,1100.798272
ATU,61.683035,23.837785
CATE(re75=0),1318.373215,1295.010415
CATE(re75!=0),-85.593549,-122.382722
ITE(10%),-2100.834974,-2124.746732
ITE(25%),-1055.282733,-1104.890381
ITE(50%),74.284460,3.468410
ITE(75%),1169.144467,1207.001699
ITE(90%),2123.485506,2067.350469
